In [13]:
# the main loop in the fine-tuning process
import os
from datasets import load_dataset
from omegaconf import DictConfig, OmegaConf
from ft_src.sft_trainer import CustomSFTTrainer
from ft_src.sft_dataset import format_data, collate_fn, generate_description
from ft_src.model import Model
import torch

cfg = OmegaConf.load("cfg/qwen2_5-vl.yaml")
# 1. load the model
vlm_model = Model(cfg.model.model_id)
print(f"Model {cfg.model.model_id} loaded successfully, The model dytype is {vlm_model.model.dtype}.")

# 2. load the data
dataset = load_dataset(cfg.dataset.dataset_id, split='train')
data_sample = dataset[126]

original_hddl = generate_description(data_sample, vlm_model.model, vlm_model.processor)

# load the lora model
vlm_model.model.load_adapter(cfg.trainer.output_dir)
ft_hddl = generate_description(data_sample, vlm_model.model, vlm_model.processor)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model Qwen/Qwen2.5-VL-7B-Instruct loaded successfully, The model dytype is torch.bfloat16.


Repo card metadata block was not found. Setting CardData to empty.


In [14]:
import pandas as pd
from IPython.display import display, HTML

def compare_generations(base_gen, ft_gen):
    # Ensure strings are displayed with line breaks
    base_gen = base_gen.encode().decode('unicode_escape')
    ft_gen = ft_gen.encode().decode('unicode_escape')

    # Create a DataFrame
    df = pd.DataFrame({
        'Base Generation': [base_gen],
        'Fine-tuned Generation': [ft_gen]
    })

    # Style the DataFrame for multiline rendering and fixed-width formatting
    styled_df = df.style.set_table_styles([
        {
            'selector': 'td',
            'props': [
                ('text-align', 'left'),
                ('white-space', 'pre-wrap'),
                ('font-family', '"Courier New", monospace'),
                ('border', '1px solid black'),
                ('padding', '10px'),
                ('vertical-align', 'top'),
                ('width', '500px'),  # adjust width as needed
                ('overflow-wrap', 'break-word')
            ]
        },
        {
            'selector': 'th',
            'props': [
                ('text-align', 'left'),
                ('font-family', '"Courier New", monospace'),
                ('border', '1px solid black'),
                ('padding', '10px')
            ]
        }
    ])

    # Display in notebook or IPython
    display(HTML(styled_df.to_html()))

compare_generations(original_hddl[0], ft_hddl[0])


,Base Generation,Fine-tuned Generation
0,```hddl (define (problem pfile01) (:domain domain_htn) (:objects plate1 - container banana1 - food apple1 - food cup1 - container ball1 - item robot1 - robot ) (:htn :parameters () :subtasks (and (task0 (move_object plate1 wp1f)) (task1 (move_object banana1 plate1)) (task2 (move_object apple1 plate1)) ) :ordering (and ) ) (:init (at plate1 wp1s) (at banana1 wp2s) (at apple1 wp2s) (at robot1 home1) ) ) ```,```hddl (define (problem pfile01) (:domain domain_htn) (:objects plate1 - container pear1 - food banana1 - food apple1 - food cup1 - container ball1 - item robot1 - robot ) (:htn :parameters () :subtasks (and (task0 (move_object plate1 wp1f)) (task1 (move_object cup1 wp1f)) (task2 (move_object plate1 wp1f)) (task3 (move_object pear1 plate1)) (task4 (move_object banana1 plate1)) (task5 (move_object apple1 plate1)) ) :ordering (and ) ) (:init (at plate1 wp1s) (at pear1 wp2s) (at banana1 wp2s) (at apple1 wp2s) (at cup1 wp2s) (at ball1 wp2s) (at robot1 home1) ) ) ```
